<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Install-Packages-and-initialize-working-directory" data-toc-modified-id="Install-Packages-and-initialize-working-directory-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Install Packages and initialize working directory</a></span></li><li><span><a href="#Load-the-data" data-toc-modified-id="Load-the-data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Load the data</a></span></li><li><span><a href="#Convert-data-to-list-of-Tuples" data-toc-modified-id="Convert-data-to-list-of-Tuples-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Convert data to list of Tuples</a></span></li><li><span><a href="#Set-up-model-for-training" data-toc-modified-id="Set-up-model-for-training-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Set up model for training</a></span></li><li><span><a href="#Model-with-noise-in-Training-Data" data-toc-modified-id="Model-with-noise-in-Training-Data-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Model with noise in Training Data</a></span></li><li><span><a href="#Attempt-3" data-toc-modified-id="Attempt-3-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Attempt 3</a></span></li></ul></div>

## Install Packages and initialize working directory

In [1]:
!python -m spacy download en_core_web_lg

     -------------------------------------- 777.4/777.4 MB 1.9 MB/s eta 0:00:00
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [101]:
#Basics
import pandas as pd
import numpy as np
import os
import re
import time
import tqdm # replace with tqdm in pyfile
from pathlib import Path
pd.set_option("display.max_rows", None)

#Spacy
import spacy
nlp = spacy.load('en_core_web_sm')

In [3]:
workdir = 'D:\HSLU_Projects\Thesis'
if os.getcwd() != workdir:
    os.chdir(workdir)
    print(f"Working Directory is now {os.getcwd()}")
else:
    print(f"Working Directory is already set to {os.getcwd()}")

Working Directory is now D:\HSLU_Projects\Thesis


## Load the data

In [4]:
table_data = pd.read_csv("Data/training_data.csv")

In [5]:
table_data.head()

,Name,Vorname,Firma,Email
0,Ademoglu,Amara,Mobimo Management AG,amara.ademoglu@mobimo.ch
1,Aebischer,Thomas,ACUTRONIC Medical Systems AG,t.aebischer@acutronic-medical.ch
2,Aeschbach,André,Dialog Verwaltungs-Data AG,andre.aeschbach@dialog.ch
3,Agotai,Doris,FHNW,doris.agotai@fhnw.ch
4,Albert,Stefan,AMAG Automobil- und Motoren AG,stefan.albert@amag.ch


In [6]:
print(table_data.shape)

(525, 4)


## Convert data to list of Tuples

For training a custom spacy model, the data has to be prepared as a list of tuples. The first element is the string, the second is a dictionary with a nest list, and within that another tuple. This will take some preparation to achieve.

In [7]:
def table_to_tuple(df):
    finished = list()
    for col in df.columns:
        test = list()
        for item in df[col].tolist():
            last_index = len(item) - 1
            new_tuple = (item, {'entities': [(0, last_index, col)]})
            finished.append(new_tuple)
    return finished

In [8]:
train_data = table_to_tuple(table_data)

In [39]:
train_data

[('Ademoglu', {'entities': [(0, 7, 'Name')]}),
 ('Aebischer', {'entities': [(0, 8, 'Name')]}),
 ('Aeschbach', {'entities': [(0, 8, 'Name')]}),
 ('Agotai', {'entities': [(0, 5, 'Name')]}),
 ('Albert', {'entities': [(0, 5, 'Name')]}),
 ('Albiez', {'entities': [(0, 5, 'Name')]}),
 ('Alkalay', {'entities': [(0, 6, 'Name')]}),
 ('Amacher', {'entities': [(0, 6, 'Name')]}),
 ('Amacker', {'entities': [(0, 6, 'Name')]}),
 ('Amrhein', {'entities': [(0, 6, 'Name')]}),
 ('Amstutz', {'entities': [(0, 6, 'Name')]}),
 ('Amstutz', {'entities': [(0, 6, 'Name')]}),
 ('Anderhub', {'entities': [(0, 7, 'Name')]}),
 ('Aregger', {'entities': [(0, 6, 'Name')]}),
 ('Aregger', {'entities': [(0, 6, 'Name')]}),
 ('Arioli', {'entities': [(0, 5, 'Name')]}),
 ('Arnold', {'entities': [(0, 5, 'Name')]}),
 ('Arnold', {'entities': [(0, 5, 'Name')]}),
 ('Arnold', {'entities': [(0, 5, 'Name')]}),
 ('Arnold', {'entities': [(0, 5, 'Name')]}),
 ('Auf der Maur', {'entities': [(0, 11, 'Name')]}),
 ('Auf der Maur', {'entities':

## Set up model for training

First, we need to split into train and test data

In [9]:
import random

random.shuffle(train_data)
cutoff = int(len(train_data)/10)
print(cutoff)

Val_DATA = train_data[:cutoff]
TRAIN_DATA = train_data[cutoff:]

210


In [10]:
len(TRAIN_DATA)

1890

In [71]:
import pandas as pd
import os
from tqdm import tqdm
import spacy
from spacy.tokens import DocBin

#nlp = spacy.blank("en") # load a new spacy model
nlp = spacy.load("en_core_web_sm") # load other spacy model

db = DocBin() # create a DocBin object

for text, annot in tqdm(TRAIN_DATA): # data in previous format
    doc = nlp.make_doc(text) # create doc object from text
    ents = []
    for start, end, label in annot["entities"]: # add character indexes
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents # label the text with the ents
    db.add(doc)
    
db.to_disk("./Model/train.spacy")

for text, annot in tqdm(Val_DATA): # data in previous format
    doc = nlp.make_doc(text) # create doc object from text
    ents = []
    for start, end, label in annot["entities"]: # add character indexes
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents # label the text with the ents
    db.add(doc)
    
db.to_disk("./Model/dev.spacy")

 34%|███▍      | 652/1890 [00:00<00:00, 3291.75it/s]

Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping

 53%|█████▎    | 1002/1890 [00:00<00:00, 3371.47it/s]


Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skippin

 90%|████████▉ | 1692/1890 [00:00<00:00, 2722.99it/s]

...


Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skippin

  0%|          | 0/210 [00:00<?, ?it/s]

Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity

100%|██████████| 210/210 [00:00<00:00, 2375.79it/s]


Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skippin

In [72]:
ner_model = spacy.load(r"./Model/output/model-last")
doc = ner_model("My name is Thomas Oliver and I work at Parashift. My email is thomas@parashift.ch")

In [74]:
spacy.displacy.render(doc, style="ent", jupyter=True)

This model did not work, evidently more noise needs to be added to the training data so it can learn what to ignore.

## Model with noise in Training Data

I will generate random noise to be added to each tuple.

In [75]:
random_vorname = ['Hello, my name is {}',
                 'You can call me {}',
                 '{} is my name',
                 'Yes, {} is how you pronounce it',
                 'No, my name is {}']
random_name = ['{} is my family name',
              'Yes, {} is my surname',
              "My surname is of course {}",
              "No it's {}",
              "Indeed, it is {}"]
random_firma = ['My company is called {}',
               "{} is where I work",
               "I am currently working at {}",
               "Yes, i work at {}, what do you want to know about it?",
               "I've changed jobs, i'm now working at {}"]
random_email = ['Yes, my email is {}',
                "You can reach me at {}",
                "The email is {}, did you get that?",
                "No, that is the wrong email, it should be {}",
                "{} is my email, please get in touch"
               ]

In [85]:
import random
import re

In [94]:
def table_to_tuple_v2(df):
    finished = list()
    term_dict = {'Name': random_name,
                "Vorname": random_vorname,
                "Firma": random_firma,
                "Email": random_email}
    for col in df.columns:
        for item in df[col].tolist():
            random_sentence = random.choice(term_dict[col])
            random_sentence = random_sentence.replace("{}", item)
            start_index = random_sentence.find(item)
            end_index = start_index + len(item)
            new_tuple = (random_sentence, {'entities': [(start_index, end_index, col)]})
            finished.append(new_tuple)
    return finished

In [95]:
tuples = table_to_tuple_v2(table_data)
tuples[:5]

[('Indeed, it is Ademoglu', {'entities': [(14, 22, 'Name')]}),
 ('Indeed, it is Aebischer', {'entities': [(14, 23, 'Name')]}),
 ('My surname is of course Aeschbach', {'entities': [(24, 33, 'Name')]}),
 ('Indeed, it is Agotai', {'entities': [(14, 20, 'Name')]}),
 ('Albert is my family name', {'entities': [(0, 6, 'Name')]})]

In [98]:
random.shuffle(tuples)
cutoff = int(len(tuples)/10)

VAL_DATA = tuples[:cutoff]
TRAIN_DATA = tuples[cutoff:]

210
1890


In [99]:
#nlp = spacy.blank("en") # load a new spacy model
nlp = spacy.load("en_core_web_sm") # load other spacy model

db = DocBin() # create a DocBin object

for text, annot in tqdm(TRAIN_DATA): # data in previous format
    doc = nlp.make_doc(text) # create doc object from text
    ents = []
    for start, end, label in annot["entities"]: # add character indexes
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents # label the text with the ents
    db.add(doc)
    
db.to_disk("./Model/train.spacy")

for text, annot in tqdm(Val_DATA): # data in previous format
    doc = nlp.make_doc(text) # create doc object from text
    ents = []
    for start, end, label in annot["entities"]: # add character indexes
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents # label the text with the ents
    db.add(doc)
    
db.to_disk("./Model/dev.spacy")

100%|██████████| 210/210 [00:00<00:00, 3333.41it/s]

Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping

In [113]:
ner_model = spacy.load(r"./Model/output/model-last")
doc = ner_model("My name is Thomas Baumgartner and I work at Parashift. My email is thomas@parashift.ch")

In [114]:
spacy.displacy.render(doc, style="ent", jupyter=True)

This model is performing much better, with only a slight issue with names. However, it can successfuly pick out "Firma" and "Email" quite well. This should be sufficient for purposes.

## Attempt 3

In [57]:
from essential_generators import DocumentGenerator
from tqdm import tqdm
gen = DocumentGenerator()


In [110]:
def table_to_tuple_v3(df):
    finished = list()
    for i in tqdm.notebook.tqdm(range(50)):
        for col in tqdm.notebook.tqdm(df.columns):
            for item in tqdm.notebook.tqdm(df[col].tolist()):
                random_sentence = gen.sentence()
                range_sentence = len(random_sentence)
                insert_index = random.randrange(range_sentence)
                res = random_sentence[ : insert_index] + ' ' + item + ' ' + random_sentence[insert_index : ]
                start_index = insert_index + 1
                end_index = start_index + len(item)
                new_tuple = (res, {'entities': [(start_index, end_index, col)]})
                finished.append(new_tuple)
    return finished

In [111]:
tuples = table_to_tuple_v3(table_data)

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/525 [00:00<?, ?it/s]

  0%|          | 0/525 [00:00<?, ?it/s]

  0%|          | 0/525 [00:00<?, ?it/s]

...

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/525 [00:00<?, ?it/s]

  0%|          | 0/525 [00:00<?, ?it/s]

  0%|          | 0/525 [00:00<?, ?it/s]

  0%|          | 0/525 [00:00<?, ?it/s]

In [112]:
random.shuffle(tuples)
cutoff = int(len(tuples)/5)

VAL_DATA = tuples[:cutoff]
TRAIN_DATA = tuples[cutoff:]

In [113]:
len(VAL_DATA)

21000

In [115]:
nlp = spacy.load("en_core_web_lg") # load other spacy model

db = DocBin() # create a DocBin object

for text, annot in tqdm.notebook.tqdm(TRAIN_DATA): # data in previous format
    doc = nlp.make_doc(text) # create doc object from text
    ents = []
    for start, end, label in annot["entities"]: # add character indexes
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents # label the text with the ents
    db.add(doc)
    
db.to_disk("./Model/train.spacy")

for text, annot in tqdm.notebook.tqdm(VAL_DATA): # data in previous format
    doc = nlp.make_doc(text) # create doc object from text
    ents = []
    for start, end, label in annot["entities"]: # add character indexes
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents # label the text with the ents
    db.add(doc)
    
db.to_disk("./Model/dev.spacy")

  0%|          | 0/84000 [00:00<?, ?it/s]

  0%|          | 0/21000 [00:00<?, ?it/s]

Step-3 : Creating the Config file for training the model
https://spacy.io/usage/training#config

Head on to Quickstart section of the page and select your configuration. (Make sure you mark NER, you can select the language based on your requirement)
image.png

Once the configurations are set, you can download it.
image.png

Step-4 : Filling the Config file
Go to the directory where base_config.cfg is stored and then run this command

python -m spacy init fill-config base_config.cfg config.cfg

Step-5 : Run the Training configuration
python -m spacy train config.cfg --output ./output --paths.train ./train.spacy --paths.dev ./dev.spacy

In [60]:
ner_model = spacy.load(r"./Model/output/model-last")
doc = ner_model("My name is Thomas Baumgartner and I work at Parashift. My email is thomas@parashift.ch")
spacy.displacy.render(doc, style="ent", jupyter=True)